In [1]:
from opf_ensemble import OpfSemble
import numpy as np
import sys

from sklearn.metrics import accuracy_score, recall_score, f1_score
from opfython.models.unsupervised import UnsupervisedOPF

opf_ens = OpfSemble(n_models=10, n_folds=10)

np.set_printoptions(threshold=sys.maxsize)

Creating model 1
Creating model  2
Creating model  3
Creating model  4
Creating model  5
Creating model  6
Creating model  7
Creating model  8
Creating model  9
Creating model  10


In [ ]:
def validation(classifiers_feats)
    k_max = [5,10,20,30,40,50]
    best_k = 0
    value_best_k = -1.0

    for k in k_max:
        opf_ens.fit_meta_model(classifiers_feats,k)
        value = f1_score(opf_ens.predict(X_valid),y_valid)
        #save validation
        if value>value_best_k:
            best_k = k
            value_best_k = value        
    return best_k

In [12]:
def run(classifiers_feats):
    best_k = validation()
    opf_ens.fit_meta_model(classifiers_feats,best_k)
    return opf_ens.predict(X_test)

5


In [ ]:
def saveResults(X_train,Y_train, X_test, Y_test,  ds,f, approach, minority_class, exec_time, path_output):

    path = '{}/{}/{}/{}'.format(path_output,approach,ds,f)
    if not os.path.exists(path):
        os.makedirs(path)
        

    results_print=[]
    accuracy, recall, f1, pred = self.classify(X_train,Y_train, X_test, Y_test, minority_class)
    results_print.append([0,accuracy, recall, f1, exec_time])

    np.savetxt('{}/pred.txt'.format(path), pred, fmt='%d')
    np.savetxt('{}/results.txt'.format(path), results_print, fmt='%d,%.5f,%.5f,%.5f,%.5f')
    print('Results:')
    print('    {}/results.txt'.format(path))

In [ ]:
datasets = ['vertebral_column']

for ds in datasets:
    for f in range(1,21):
        
        train = np.loadtxt('data/{}/{}/train.txt'.format(ds,f),delimiter=',', dtype=np.float32)
        valid = np.loadtxt('data/{}/{}/valid.txt'.format(ds,f),delimiter=',', dtype=np.float32)
        test = np.loadtxt('data/{}/{}/test.txt'.format(ds,f),delimiter=',', dtype=np.float32)

        X = train[:,:-1]
        y = train[:,-1].astype(np.int) 

        X_valid = valid[:,:-1]
        y_valid = valid[:,-1].astype(np.int) 

        X_test = test[:,:-1]
        y_test = test[:,-1].astype(np.int) 

        new_x = opf_ens.fit(X, y)
        pred_ensamble = run(new_x, X_test)
        
        #save results
        print('{}={}'.format('Ensamble',f1_score(pred_ensamble,y_test)))
        for key in opf_ens.ensemble:
            model = opf_ens.ensemble[key]
            pred = model.predict(X_test)
            #res = (yhat == test_y).astype(int)
            print('{}={}'.format(key,f1_score(pred,y_test))) 